In [1]:
from io_minio import get_parquet_minio
from db_common import insert_data, select_data
import pandas as pd
import cv2
import numpy as np
import tempfile
import multiprocessing
from sklearn.neighbors import NearestNeighbors
from io_minio import get_image_minio, upload_img_path, upload_parquet
from libs.knn_process import knn_process_df_image
import pyarrow.dataset as ds
import gc
from db_common import insert_data, select_data
from common import generate_hash
import io
from multiprocessing import cpu_count
import boto3
import datetime
from concurrent.futures import ProcessPoolExecutor, as_completed, ThreadPoolExecutor
from tqdm import tqdm
import random
from multiprocessing import Pool, cpu_count
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd
import os
import pandas as pd
import numpy as np
import cv2
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [2]:
nots_id_product = []

id_product_query = f"WHERE pd.ID_PRODUCT NOT IN ({','.join(str(x) for x in nots_id_product)})" if len(nots_id_product) > 0 else ''
df_result = select_data(f"SELECT pd.ID_PRODUCT, pd.ID_PRODUCT_DATA, d.PATH_DATA FROM PRODUCT_DATA pd JOIN DATA d ON d.ID_DATA = pd.ID_DATA {id_product_query}")

df_all_results = []

for idx, row  in df_result.iterrows():
    df_all_results.append(get_parquet_minio(row['path_data']))

df_all_results = pd.concat(df_all_results, ignore_index=True)
df_all_results.to_parquet('df_all_results.parquet')

In [3]:
df_all_results = pd.read_parquet('df_all_results.parquet')
# df_all_results

In [4]:
df_results = df_all_results[~df_all_results['id_product'].isin([1])]
vector_columns = df_results.columns
vector_columns = [i for i in vector_columns if i not in [ 'id_product', 'id_product_data', 'path_data']]
vector_columns

['image_process',
 'img_rgb',
 'img_cinza',
 'img_suavizada',
 'img_bordas_canny',
 'mascara_segmentada',
 'contornos',
 'altura_img',
 'largura_img',
 'contornos_filtrados',
 'mascara_final',
 'img_com_contornos',
 'metricas_geo',
 'vetor_hog',
 'img_visual_hog',
 'img_visual_lbp',
 'hist_lbp',
 'metricas_glcm']

### Daqui pra baixo não funciona

In [5]:
# --- Função de Processamento (Apenas Concatena, SEM Padding) ---
def concat_row_vector(row, vector_columns):
    """
    Concatena os vetores de features da linha.
    """
    parts = []
    for col in vector_columns:
        vec = row[col]
        if isinstance(vec, list) or isinstance(vec, np.ndarray):
            parts.append(np.array(vec, dtype=np.float32))
        elif pd.api.types.is_numeric_dtype(type(vec)):
             parts.append(np.array([vec], dtype=np.float32))
             
    return np.concatenate(parts)

# --- 1️⃣ Geração Otimizada da Matriz de Features para Treinamento ---

# Passo A: Calcular o max_len e gerar a lista de vetores brutos
print("Passo 1: Calculando max_len e concatenando vetores...")
raw_vectors_list = []
max_len = 0
total_rows = len(df_results)

# Itera sobre as linhas para concatenar e rastrear o maior comprimento
for index in range(total_rows):
    row = df_results.iloc[index]
    # Usa a função de concatenação simplificada
    concatenated_vec = concat_row_vector(row, vector_columns)
    
    raw_vectors_list.append(concatenated_vec)
    
    current_len = len(concatenated_vec)
    if current_len > max_len:
        max_len = current_len

print(f"Tamanho máximo do vetor de feature (max_len): {max_len}")
print(f"Total de linhas a processar: {total_rows}")


# Passo B & C (COMBINADOS): Criar Feature Matrix diretamente com Padding/Truncamento
print("Passo 2: Criando Feature Matrix com padding/truncamento...")

# Pré-aloca a matriz de features - este é o ponto chave para economia de RAM,
# pois evita o vstack ao final.
feature_matrix = np.zeros((total_rows, max_len), dtype=np.float32)

for i, v in enumerate(raw_vectors_list):
    v_len = len(v)
    
    if v_len <= max_len:
        # Aplica padding se o vetor for menor
        feature_matrix[i, :v_len] = v
    else:
        # Aplica truncamento se o vetor for maior
        feature_matrix[i, :] = v[:max_len]

# Libera a lista de vetores brutos que não são mais necessários
del raw_vectors_list 

print(f"Feature matrix shape: {feature_matrix.shape}")


# --- 2️⃣ Treinar KNN ---
# n_neighbors pode ser ajustado, usar len(df_results) retorna todos os vizinhos
knn = NearestNeighbors(n_neighbors=min(len(df_results), 10), metric="euclidean") 
knn.fit(feature_matrix)

Passo 1: Calculando max_len e concatenando vetores...
Tamanho máximo do vetor de feature (max_len): 161862
Total de linhas a processar: 6866
Passo 2: Criando Feature Matrix com padding/truncamento...
Feature matrix shape: (6866, 161862)


,n_neighbors,10
,radius,1.0
,algorithm,'auto'
,leaf_size,30
,metric,'euclidean'
,p,2
,metric_params,None
,n_jobs,None


In [6]:
def process_and_pad_vector(row, vector_columns, max_len=None, dtype=np.float32):
    """
    Concatena vetores muito grandes de forma eficiente.
    - Converte cada coluna para np.ndarray dtype=float32
    - Concatena com mínimo de cópias
    - Aplica padding ou truncamento se max_len for informado
    """

    # Preparar lista de vetores convertidos para ndarray
    parts = []
    total_len = 0

    for col in vector_columns:
        vec = row[col]

        # Ignorar se for None
        if vec is None:
            continue

        # Converte sem copiar quando possível
        if isinstance(vec, np.ndarray):
            arr = vec.astype(dtype, copy=False)
        elif isinstance(vec, list):
            arr = np.asarray(vec, dtype=dtype)
        else:
            # Se ainda assim for escalar, ignora ou converte para vetor de 1 elemento
            arr = np.array([vec], dtype=dtype)

        parts.append(arr)
        total_len += arr.shape[0]

    # Caso sem padding: concatenação única
    concatenated = np.concatenate(parts) if len(parts) > 1 else parts[0]

    # Aplicar padding ou truncamento
    if max_len is not None:
        if total_len < max_len:
            # Prealocar vetor final — mais rápido que np.pad
            padded = np.zeros(max_len, dtype=dtype)
            padded[:total_len] = concatenated
            return padded

        elif total_len > max_len:
            return concatenated[:max_len]

    return concatenated


In [10]:
df_img_test = pd.DataFrame([knn_process_df_image(image_path='../dataset_test/Apple/3.jpg')])
df_img_test

query_vec = process_and_pad_vector(
    df_img_test.iloc[0],
    vector_columns,
    max_len=max_len
).reshape(1, -1)

query_vec

array([[253., 253., 253., ..., 251., 251., 251.]],
      shape=(1, 161862), dtype=float32)